CS524: Introduction to Optimization Lecture 19
======================================

## Michael Ferris<br> Computer Sciences Department <br> University of Wisconsin-Madison

## October 16, 2024
--------------

#  Gandhi (Winston, p 463)

A clothing manufacturer can make shirts, shorts, and pants. A different machine must be rented to make each of these three items. A shirt machine costs \\$200/month to rent, while a shorts machine costs \\$150/month and a pants machine costs \\$100/month. (The manufacturer can choose not to rent a shirt machine, for example, but in this case he cannot make any shirts.) Each shirt requires 3 hours of labor and 4 square yards of cloth; each pair of pants requires 6 hours of labor and 4 square yards of cloth; each pair of shorts requires 2 hours of labor and 3 square yards of cloth. The total amount of labor available is 150 hours, and there are 160 square yards of cloth.

Each shirt sells for \\$12 and costs \\$6 to make. Each pair of shorts sells for \\$8 and costs \\$4 to make, while each pair of pants sells for \\$15 and costs \\$8 to make.

Determine which item(s) should be manufactured, and how many of each.

In [1]:
from gamspy import Container, Sense, Problem, Sum, Options, Number
import numpy as np
import sys

options = Options(variable_listing_limit=0, equation_listing_limit=0,relative_optimality_gap=0.)
m = Container(options=options,debugging_level='keep')

In [3]:
# DATA
item = m.addSet('item',records=['shirt', 'shorts', 'pants'])
input = m.addSet('input',records=[('labor',"hours available per week"),
    ('cloth',"sq yards available per week")])
revcost = m.addSet('revcost',records=[('price',"selling price in dollars"),
    ('var-cost',"variable cost in dollars")])
rentmach = m.addParameter('rentmach',domain=item,description="cost of rental in dollars/week",
    records = [('shirt',200),('shorts',150), ('pants',100)])
resources = m.addParameter('resources',domain=input,
    records = [('labor',150), ('cloth',160)])
requirements=m.addParameter('requirements',domain=[item,input],description='Table 2 from Winston',
    records=np.array([[3, 4], [2, 3], [6, 4]]))
revenue=m.addParameter('revenue',domain=[item,revcost],description='Table 3 from Winston',
    records=np.array([[12, 6], [8, 4], [15, 8]]))
bigM = m.addParameter('bigM', records=1000)

# MODEL

amount = m.addVariable('amount','positive',domain=item)
useMachine = m.addVariable('useMachine','binary',domain=item)
profit = m.addVariable('profit','free')

EQresources = m.addEquation('EQresources',domain=input)
EQresources[input]= (
    Sum(item,requirements[item,input]*amount[item]) <= resources[input])

EQrental = m.addEquation('EQrental',domain=item)
EQrental[item]= (
    amount[item] <= useMachine[item]*bigM)

objective = m.addEquation('objective')
objective[:]= (profit == 
    Sum(item, revenue[item,'price']*amount[item])
    - Sum(item, revenue[item,'var-cost']*amount[item]) 
    - Sum(item, rentmach[item] * useMachine[item]))

fixedCost = m.addModel('fixedCost',
    equations=[EQresources,EQrental,objective],
    problem=Problem.MIP,
    sense=Sense.MAX,
    objective=profit,
)

,item,value
0,shirt,200.0
1,shorts,150.0
2,pants,100.0


Set the relative optimality tolerance (gap) to 0, then solve bigM problem:

In [3]:
fixedCost.solve(output=None)
print(f"amount =\n {amount.records[['item','level']]}\nuse machines =\n {useMachine.records[['item','level']]}")

amount =
      item  level
0   shirt    0.0
1  shorts    0.0
2   pants   25.0
use machines =
      item  level
0   shirt    0.0
1  shorts    0.0
2   pants    1.0


Alternative: Use cplex with indicator constraints (turn on EQrental2 if useMachine = 0)

In [4]:
EQrental2 = m.addEquation('EQrental2',domain=[item])
EQrental2[item] = (amount[item] == 0)

fixedCost = m.addModel('fixedCost',
    equations=[EQresources,EQrental2,objective],
    problem=Problem.MIP,
    sense=Sense.MAX,
    objective=profit,
)

fixedCost.solve(output=None,solver='cplex',solver_options={'indic': 'EQrental2(item)$useMachine(item) 0'})
print(f"amount =\n {m['amount'].records[['item','level']]}\nuse machines =\n {m['useMachine'].records[['item','level']]}")

amount =
      item  level
0   shirt    0.0
1  shorts    0.0
2   pants   25.0
use machines =
      item  level
0   shirt    0.0
1  shorts    0.0
2   pants    1.0


Modify problem to have different costs, capacities and multiple machines

In [5]:
rentmach.setRecords([ ('shirt',30), ('shorts',50), ('pants',20) ])
capacityMachine = m.addParameter('capacityMachine',domain=item,
    records=[ ('shirt',10), ('shorts',10), ('pants',8) ])
    
numberMachines = m.addVariable('numberMachines','integer',domain=item)

# No change to declaration of equations, just definition
EQrental[item]= (
    amount[item] <= numberMachines[item]*capacityMachine[item] )

objective[:]= (
    profit ==   Sum(item, revenue[item,'price']*amount[item])
        - Sum(item, revenue[item,'var-cost']*amount[item]) 
        - Sum(item, rentmach[item]* numberMachines[item]) )

fixedCost = m.addModel('fixedCost',
    equations=[EQresources,EQrental,objective],
    problem=Problem.MIP,
    sense=Sense.MAX,
    objective=profit,
)

fixedCost.solve(solver="cplex", solver_options={'*': 'no option'})
print(f"amount =\n {amount.records[['item','level']]}\nnumber machines =\n {numberMachines.records[['item','level']]}")

amount =
      item  level
0   shirt   30.0
1  shorts    0.0
2   pants    8.0
number machines =
      item  level
0   shirt    3.0
1  shorts    0.0
2   pants    1.0
